In [41]:
import pandas as pd
import numpy as np

## Identify Overlapping Peaks of Two Peak Tables

In [42]:
def ppm_calculation(x, y):
    return abs(x-y)*10e6/y

In [50]:
def nearest_value(lst, K):
    return lst[min(range(len(lst)), key = lambda i: abs(lst[i]-K))]

In [43]:
def n_overlapping_peaks_v0(t_1, t_2, col_name_mz, col_name_rt, ppm_threshold, rt_tolerance):
    # Calculating number of overlapping peaks between two peak tables. Make sure t_2 has less rows than t_1
    n = 0
    for i in range(len(t_2)):
        mz = t_2[col_name_mz].iloc[i]
        rt = t_2[col_name_rt].iloc[i]
        if len(t_1[(ppm_calculation(mz, t_1[col_name_mz])<=ppm_threshold) & (abs(table_1[col_name_rt] - rt) < rt_tolerance)])>0:
            n += 1
    return n

In [78]:
def overlapping_peaks(t_1, t_2, col_name_mz, col_name_rt, ppm_threshold, rt_tolerance):
    flag = 0
    flagged_rows = []
    for i in range(len(t_2)):
        second_flag = 0
        mz_list = list(table_1[col_name_mz].str.split("_", expand=True)[0]) + [x for x in list(table_1[col_name_mz].str.split("_", expand=True)[1]) if str(x)!="None"]
        rt_list = list(table_1[col_name_rt].str.split("_", expand=True)[0]) + [x for x in list(table_1[col_name_rt].str.split("_", expand=True)[1]) if str(x)!="None"]
        mz_list = [float(x) for x in mz_list]
        rt_list = [float(x) for x in rt_list]
        if (type(t_2[col_name_mz].iloc[i]) == float) or ("_" not in t_2[col_name_mz].iloc[i]):
            mz = float(t_2[col_name_mz].iloc[i])
            rt = float(t_2[col_name_rt].iloc[i])
            nearest_id = mz_list.index(nearest_value(mz_list, mz))
            if (ppm_calculation(mz, mz_list[nearest_id]) <= ppm_threshold) and (abs(rt - rt_list[nearest_id]) < rt_tolerance):
                continue
            else:
                flag = 1
        else:
            for j in range(len(t_2[col_name_mz].iloc[i].split("_"))):
                mz = float(t_2[col_name_mz].iloc[i].split("_")[j])
                rt = float(t_2[col_name_rt].iloc[i].split("_")[j])
                if (ppm_calculation(mz, mz_list[nearest_id]) <= ppm_threshold) and (abs(rt - rt_list[nearest_id]) < rt_tolerance):
                    second_flag = 1
                    continue
            if second_flag == 0:
                flag = 1
        if flag == 1:
            flagged_rows.append(i)
    return t_2.drop(labels=flagged_rows, axis=0)

### Local CMD VS HPC CMD

In [61]:
table_1 = pd.read_csv("../manuscript/peak_table/hpc_cmd/ms-dial/AlignResult-20231272341.txt", delimiter="\t", header=4)

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [62]:
table_2 = pd.read_csv("../manuscript/peak_table/local_cmd/ms-dial/AlignResult-2023129133.txt", delimiter="\t", header=4)

In [63]:
len(table_1)

25951

In [64]:
len(table_2)

25445

In [65]:
n_overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

18857

#### tests

In [45]:
table_1 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/hpc_cmd/ms-flo/AlignResult-2023282236_processed.txt", delimiter="\t")

In [46]:
table_2 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/local_cmd/ms-flo/AlignResult-2023291844_processed.txt", delimiter="\t")

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [47]:
table_1.head()

,identifier,Alignment ID,Average Mz,Average Rt(min),duplicate_flag,adduct_flag,isotope_flag,isotope_phr,Metabolite name,Adduct type,...,SB3,SA2,SB4,SB5,SA1,SB2,SA3,SA4,SB1,SA5
0,0.03_171.99,1886,171.9936,0.031,NaN,NaN,NaN,NaN,Unknown,[M+H]+,...,3555931.0,3107714.50,3700617.8,4714087.50,4075218.80,3473687.0,3224671.2,3100795.20,4109884.80,3218963.5
1,0.03_115.11,507,115.1119,0.034,NaN,NaN,NaN,NaN,Unknown,[M+H]+,...,396968.0,441166.22,386949.2,507670.12,518073.84,416492.8,406312.0,442892.34,425433.44,421556.3
2,0.04_182.99,2206,182.9855,0.038,NaN,NaN,NaN,NaN,Unknown,[M+H]+,...,6653673.5,6435527.50,6556864.5,7956248.50,8745378.00,6570529.0,6564399.0,6728224.00,8370828.00,6689435.0
3,0.04_196.15,2621,196.1549,0.042,NaN,NaN,NaN,NaN,Unknown,[M+H]+,...,2228402.2,2745459.00,2217159.0,2885036.20,3511845.20,2311969.0,2600677.0,2587100.20,2931167.50,2586849.2
4,0.43_118.09,581,118.0864,0.427,Match #069: Possible duplicate of 0.52_118.09 ...,NaN,NaN,NaN,w/o MS2:Valine; LC-ESI-QTOF; MS2; CE,[M+H]+,...,46561192.0,47708448.00,46621160.0,44220552.00,57559904.00,47149300.0,47418344.0,46128584.00,57822920.00,47533796.0


In [48]:
table_2.head()

,identifier,Alignment ID,Average Mz,Average Rt(min),duplicate_flag,adduct_flag,isotope_flag,isotope_phr,Metabolite name,Adduct type,...,SA1,SA2,SA3,SA4,SA5,SB1,SB2,SB3,SB4,SB5
0,0.03_171.99,1848,171.9936,0.031,NaN,NaN,NaN,NaN,Unknown,[M+H]+,...,4075218.80,3107714.5,3224671.20,3100795.20,3218963.50,4109884.80,3473687.00,3555931.00,3700617.80,4714087.50
1,0.04_182.99,2151,182.9855,0.038,NaN,NaN,NaN,NaN,Unknown,[M+H]+,...,8745378.00,6435527.5,6564399.00,6728224.00,6689435.00,8370828.00,6570529.00,6653673.50,6556864.50,7956248.50
2,0.04_196.15,2554,196.1549,0.042,NaN,NaN,NaN,NaN,Unknown,[M+H]+,...,3511845.20,2745459.0,2600677.00,2587100.20,2586849.20,2931167.50,2311969.00,2228402.20,2217159.00,2885036.20
3,0.41_118.44,554,118.4398,0.407,NaN,NaN,NaN,NaN,Unknown,[M+H]+,...,35294.02,30813.0,50259.05,24106.03,33674.96,31959.85,25801.86,21137.22,19737.76,18970.89
4,0.43_118.09,539,118.0864,0.427,Match #065: Possible duplicate of 0.52_118.09 ...,NaN,NaN,NaN,w/o MS2:Valine; LC-ESI-QTOF; MS2; CE,[M+H]+,...,57559904.00,47708448.0,47418344.00,46128584.00,47533796.00,57822920.00,47149300.00,46561192.00,46621160.00,44220552.00


In [79]:
table_merged = overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

In [80]:
len(table_merged)

17245

In [55]:
table_1["Average Mz"].str.split("_", expand=True)

,0,1
0,171.9936,None
1,115.1119,None
2,182.9855,None
3,196.1549,None
4,118.0864,None
...,...,...
25673,157.0355,None
25674,138.0586,None
25675,157.0354,None
25676,157.0354,None


In [21]:
len(list(table_1["Average Mz"].str.split("_", expand=True)[0]) + list(table_1["Average Mz"].str.split("_", expand=True)[1]))

51356

In [63]:
mzs = [float(x) for x in list(table_1["Average Mz"].str.split("_", expand=True)[0]) + [x for x in list(table_1["Average Mz"].str.split("_", expand=True)[1]) if str(x)!="None"]]

In [64]:
rts = [float(x) for x in list(table_1["Average Rt(min)"].str.split("_", expand=True)[0]) + [x for x in list(table_1["Average Rt(min)"].str.split("_", expand=True)[1]) if str(x)!="None"]]

In [65]:
mzs

[171.9936,
 115.1119,
 182.9855,
 196.1549,
 118.0864,
 144.1384,
 176.1399,
 260.2221,
 199.9885,
 213.983,
 141.1276,
 135.5307,
 227.983,
 219.9724,
 154.9902,
 139.9824,
 186.9568,
 110.0204,
 139.1114,
 131.9743,
 132.5418,
 105.9631,
 172.9569,
 113.9636,
 112.0178,
 196.0171,
 224.0116,
 141.9584,
 270.9645,
 319.9156,
 198.011,
 400.9232,
 182.9856,
 401.875,
 302.9152,
 349.9324,
 276.0271,
 266.0173,
 442.8981,
 348.951,
 422.8895,
 125.525,
 112.539,
 207.9849,
 255.0215,
 375.8752,
 127.5444,
 245.9159,
 107.031,
 376.0199,
 114.5586,
 131.1181,
 164.0673,
 188.0234,
 166.4216,
 260.9244,
 286.9426,
 329.9467,
 141.0409,
 218.0339,
 317.9804,
 180.9841,
 310.9077,
 153.033,
 169.0459,
 204.0006,
 151.0354,
 347.9925,
 505.8317,
 205.9963,
 306.9652,
 316.9147,
 197.041,
 507.8716,
 100.0144,
 120.5276,
 222.0108,
 416.9006,
 192.0619,
 173.0785,
 122.5471,
 101.1074,
 245.0274,
 250.0062,
 129.5331,
 124.5437,
 109.5216,
 162.9737,
 131.5527,
 194.0595,
 401.4015,
 284.9873

In [66]:
rts

[0.031,
 0.034,
 0.038,
 0.042,
 0.427,
 0.448,
 0.449,
 0.457,
 0.462,
 0.467,
 0.467,
 0.469,
 0.475,
 0.475,
 0.476,
 0.477,
 0.477,
 0.477,
 0.477,
 0.478,
 0.479,
 0.479,
 0.48,
 0.481,
 0.482,
 0.483,
 0.485,
 0.487,
 0.487,
 0.488,
 0.488,
 0.489,
 0.489,
 0.489,
 0.491,
 0.493,
 0.493,
 0.493,
 0.493,
 0.494,
 0.494,
 0.494,
 0.494,
 0.495,
 0.495,
 0.496,
 0.496,
 0.496,
 0.496,
 0.497,
 0.497,
 0.497,
 0.498,
 0.498,
 0.498,
 0.498,
 0.499,
 0.499,
 0.499,
 0.499,
 0.499,
 0.499,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.501,
 0.501,
 0.501,
 0.501,
 0.501,
 0.501,
 0.501,
 0.501,
 0.501,
 0.501,
 0.501,
 0.502,
 0.502,
 0.502,
 0.502,
 0.502,
 0.502,
 0.502,
 0.502,
 0.502,
 0.502,
 0.502,
 0.502,
 0.502,
 0.502,
 0.502,
 0.502,
 0.502,
 0.502,
 0.502,
 0.502,
 0.502,
 0.502,
 0.502,
 0.502,
 0.502,
 0.502,
 0.503,
 0.503,
 0.503,
 0.503,
 0.503,
 0.503,
 0.503,
 0.503,
 0.503,
 0.503,
 0.503,
 0.503,
 0.503,
 0.

In [67]:
zip(mzs, rts)

In [60]:
len(mzs)

25948

In [61]:
len(rts)

25948

In [34]:
a = ["1", "2", "3"]

In [40]:
type(table_1["Average Mz"].iloc[0])==float

False

In [68]:
a

[1, 2, 23, 3, 3, 4, 5, 5, 3, 2, 2, 34]

In [70]:
a.index(2)

1

In [36]:
[float(x) for x in a]

[1.0, 2.0, 3.0]

### HPC NF VS Local NF

In [51]:
table_1 = pd.read_csv("../manuscript/peak_table/hpc_nf/ms-dial/AlignResult-202212142025.txt", delimiter="\t", header=4)

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [52]:
table_2 = pd.read_csv("../manuscript/peak_table/local_nf/ms-dial/AlignResult-20231271922.txt", delimiter="\t", header=4)

In [53]:
len(table_1)

25951

In [54]:
len(table_2)

25445

In [55]:
n_overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

18857

### HPC CMD VS HPC NF

In [56]:
table_1 = pd.read_csv("../manuscript/peak_table/hpc_cmd/ms-dial/AlignResult-20231272341.txt", delimiter="\t", header=4)

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [57]:
table_2 = pd.read_csv("../manuscript/peak_table/hpc_nf/ms-dial/AlignResult-202212142025.txt", delimiter="\t", header=4)

In [58]:
len(table_1)

25951

In [59]:
len(table_2)

25951

In [60]:
n_overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

25951

### Local CMD VS Local NF

In [68]:
table_1 = pd.read_csv("../manuscript/peak_table/local_cmd/ms-dial/AlignResult-2023129133.txt", delimiter="\t", header=4)

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [69]:
table_2 = pd.read_csv("../manuscript/peak_table/local_nf/ms-dial/AlignResult-20231271922.txt", delimiter="\t", header=4)

In [70]:
len(table_1)

25445

In [71]:
len(table_2)

25445

In [72]:
n_overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

25445

### Overlapping Between Local and Benchmark

In [103]:
table_1 = pd.read_csv("../manuscript/peak_table/local_nf/ms-dial/AlignResult-20231271922.txt", delimiter="\t", header=4)

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [104]:
table_2 = pd.read_excel("../manuscript/benchmark_libs/lib.xlsx", sheet_name="QE HF dataset_TableS8", header=6)

In [105]:
len(table_1)

25445

In [106]:
table_1.head()

,Alignment ID,Average Rt(min),Average Mz,Metabolite name,Adduct type,Post curation result,Fill %,MS/MS assigned,Reference RT,Reference m/z,...,SA1,SA2,SA3,SA4,SA5,SB1,SB2,SB3,SB4,SB5
0,0,0.562,100.01421,Unknown,[M+2H]2+,NaN,0.3,False,NaN,NaN,...,8714982.00,7387047.5,10247660.0,8517238.0,6663123.0,7660357.00,6522387.5,6514624.0,9974121.0,9562859.0
1,1,35.967,100.01428,Unknown,[M+NH4]2+,adduct linked to 2150_[M+H]+,0.9,False,NaN,NaN,...,765282.90,942749.0,1229463.0,880391.2,1132224.9,907290.06,1239561.4,1271661.1,1253242.1,1348959.9
2,2,0.500,100.01443,Unknown,[M+H]2+,adduct linked to 3419_[M+Na]+,1.0,False,NaN,NaN,...,8001129.00,13925881.0,16331838.0,16200029.0,18298616.0,6891007.50,15492309.0,16130891.0,17702120.0,20029132.0
3,3,0.592,100.01444,Unknown,[M+2H]2+,NaN,0.9,False,NaN,NaN,...,8898810.00,10046386.0,9382828.0,10474542.0,10473924.0,12107133.00,8653832.0,9658213.0,12457828.0,11770822.0
4,4,31.647,100.06959,Unknown,[M+H]+,NaN,1.0,False,NaN,NaN,...,824087.44,1108073.5,1230883.0,1154022.8,1614246.2,2215985.00,2037453.5,2112368.2,2052877.8,1913928.4


In [107]:
table_1["Average Mz"]

0         100.01421
1         100.01428
2         100.01443
3         100.01444
4         100.06959
            ...    
25440    1490.57178
25441    1490.92615
25442    1491.03601
25443    1497.48169
25444    1497.52856
Name: Average Mz, Length: 25445, dtype: float64

In [108]:
table_1["Average Rt(min)"]

0         0.562
1        35.967
2         0.500
3         0.592
4        31.647
          ...  
25440    30.313
25441     4.751
25442     4.750
25443    31.699
25444    30.323
Name: Average Rt(min), Length: 25445, dtype: float64

In [109]:
len(table_2)

836

In [110]:
table_2.head()

,Feature number,Compound formula,Adduct,Average Mz,Average Rt(min),Fold change,p-value,Compound concentration ratio,Unnamed: 8,Unnamed: 9
0,1,C21H26N2S2,+H,371.1617,18.85,0.474166,0.000481,1/2,NaN,NaN
1,2,C25H31NO6,+H,442.2233,17.66,21.230982,0.000001,16/1,NaN,NaN
2,3,C14H17N3O2S,+H,292.1118,3.01,1.905322,0.019356,2/1,NaN,NaN
3,4,C27H31N7OS,+H,502.2395,22.78,1.062849,0.289866,1/1,NaN,NaN
4,5,C20H23NO4,+H,342.1706,3.37,1.066033,0.399052,1/1,NaN,NaN


In [111]:
n_overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

401

### Overlapping Between HPC and Benchmark

In [112]:
table_1 = pd.read_csv("../manuscript/peak_table/hpc_nf/ms-dial/AlignResult-202212142025.txt", delimiter="\t", header=4)

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [113]:
table_2 = pd.read_excel("../manuscript/benchmark_libs/lib.xlsx", sheet_name="QE HF dataset_TableS8", header=6)

In [114]:
len(table_1)

25951

In [115]:
n_overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

390